In [1]:
import random
import numpy as np
import crocoddyl
import keras
from keras.models import load_model
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
crocoddyl.switchToNumpyMatrix()


Using TensorFlow backend.
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py

In [2]:
def iteration_analysis(MAXITER:int =1000):
    """
    Script to store the number of iterations crocoddyl takes to generate a trajectory.
    Here, we generate 10K trajectories and store the number of iterations taken by  crocoddyl 
    for each trajectory
    
    """
    iterations = []
    not_feasible = 0
    
    NUM_TRAJ = 10000


    for _ in range(NUM_TRAJ):
        initial_config = [random.uniform(-2.1, 2.),
                                    random.uniform(-2.1, 2.), 
                                    random.uniform(0, 1)]

        model = crocoddyl.ActionModelUnicycle()

        model.costWeights = np.matrix([1, 0.3]).T

        problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)

        ddp = crocoddyl.SolverDDP(problem)

        done = ddp.solve([], [], MAXITER)
        if not ddp.isFeasible:
            not_feasible += 1

        else:
            iterations.append(ddp.iter)

    max_value = max(iterations)
    min_value = min(iterations)
    avg_value = sum(iterations)//len(iterations)
    print(f"Total trajectories: {NUM_TRAJ}, not feasible trajectories: {not_feasible}\n\n")
    print(f'Maximum iterations allowed : {MAXITER}\n')
    print(f"\nMinimum iterations taken -> {min_value},\
          Maximum iterations taken -> {max_value},\
          Average -> {avg_value} \n")
    print(".............................\n")
    
    return iterations

Group1 = iteration_analysis(MAXITER = 1000)
Group2 = iteration_analysis(MAXITER = 100)
Group3 = iteration_analysis(MAXITER = 500)

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 1000


Minimum iterations taken -> 2,          Maximum iterations taken -> 1000,          Average -> 26 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 100


Minimum iterations taken -> 2,          Maximum iterations taken -> 100,          Average -> 25 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 500


Minimum iterations taken -> 2,          Maximum iterations taken -> 500,          Average -> 26 

.............................



In [1]:
from data_net import *
import random
import numpy as np
import crocoddyl
from keras.models import load_model

def warmstart(n_trajectories: int = 10000, load_net: bool = False):
    if load_net:
        neuralNet = load_model('basic_model.h5')
    else:
        neuralNet = train_net()
        
    print(f" With warmstart, without warmstart ")
    
    for _ in range(n_trajectories):
            
            initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]

            model = crocoddyl.ActionModelUnicycle()
            model2 = crocoddyl.ActionModelUnicycle()
            
            model.costWeights = np.matrix([1., .3]).T
            model2.costWeights = np.matrix([1., .3]).T
    
            
            problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)
            problem2 = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model2 ] * 20, model2)

            
            ddp = crocoddyl.SolverDDP(problem)
            ddp2 = crocoddyl.SolverDDP(problem2)
            
            
            x_data = np.matrix(initial_config).reshape(1, 3)

            prediction = neuralNet.predict(x_data).reshape(20, 5)

            xs = np.matrix(prediction[:, 0:3])

            us = np.matrix(prediction[:, 3:5])
            

            ddp_xs = []
            ddp_xs.append(np.matrix(initial_config).reshape(1, 3).T)
            for _ in  range(xs.shape[0]):
                ddp_xs.append((np.matrix(xs[_]).T))

            ddp_us = []
            for _ in  range(us.shape[0]):
                ddp_us.append(np.matrix(us[_]).T)

            ddp.solve(ddp_xs,ddp_us)
            ddp2.solve()
            print("  ", ddp.iter, "  ", ddp2.iter)
                
warmstart(n_trajectories=1000, load_net=False)    

Using TensorFlow backend.
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py


 With warmstart, without warmstart 
   10    11
   21    24
   5    5
   31    39
   8    9
   14    13
   11    11
   34    29
   23    21
   15    14
   14    13
   30    26
   69    68
   26    27
   50    47
   13    12
   22    24
   10    9
   29    32
   28    32
   13    14
   18    19
   9    9
   5    5
   34    41
   22    21
   26    28
   26    33
   38    31
   17    18
   46    36
   38    42
   73    54
   10    10
   19    18
   13    15
   27    20
   17    17
   49    46
   5    5
   9    9
   33    42
   11    10
   50    39
   10    9
   15    14
   15    17
   100    100
   5    5
   22    19
   27    28
   25    27
   28    30
   31    28
   24    31
   8    8
   42    46
   17    19
   33    30
   30    33
   5    5
   28    27
   4    4
   6    6
   8    9
   16    16
   27    24
   28    36
   4    4
   24    27
   23    26
   32    26
   7    7
   6    6
   9    9
   24    26
   40    43
   24    22
   40    30
   16    15
   26    28
   5    5
   19    18
 

   26    26
   20    21
   59    37
   5    5
   26    23
   27    23
   6    6
   81    82
   35    35
   23    26
   100    97
   19    17
   22    25
   46    51
   24    28
   5    5
   5    4
   38    24
   33    40
   24    23
   47    38
   13    12
   7    7
   80    78
   36    27
   27    26
   15    17
   100    79
   36    41
   6    6
   9    9
   4    5
   36    43
   39    33
   35    32
   5    5
   18    20
   15    14
   11    11
   25    28
   29    24
   6    6
   44    41
   9    9
   72    71
   9    9
   30    27
   6    6
   29    23
   31    33
   7    6
   8    8
   4    4
   57    67
   36    26
   42    42
   5    5
   4    4
   64    44
   39    53
   11    11
   6    5
   15    16
   22    20
   43    54
   20    21
   43    48
   31    33
   32    30
   6    6
   51    47
   29    24
   4    4
   8    7
   13    14
   75    65
   26    26
   9    9
   11    11
   25    31
   100    100
   37    35
   59    54
   20    19
   63    59
   20    21
   13    1

In [2]:
initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]

model = crocoddyl.ActionModelUnicycle()
model2 = crocoddyl.ActionModelUnicycle()

model.costWeights = np.matrix([1., .3]).T
model2.costWeights = np.matrix([1., .3]).T


problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)
problem2 = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model2 ] * 20, model2)


ddp = crocoddyl.SolverDDP(problem)
ddp2 = crocoddyl.SolverDDP(problem2)

            

In [5]:
ddp.solve()

True

In [6]:
ddp.xs

[matrix([[0.24469323],
         [1.0182596 ],
         [0.17483822]]), matrix([[0.31938757],
         [1.03145375],
         [0.52803402]]), matrix([[0.12755307],
         [0.91956068],
         [0.72216947]]), matrix([[-0.119221  ],
         [ 0.70217405],
         [ 0.71639857]]), matrix([[-0.33243106],
         [ 0.51652862],
         [ 0.56796566]]), matrix([[-0.47034728],
         [ 0.42852399],
         [ 0.35975872]]), matrix([[-0.5049535 ],
         [ 0.41550764],
         [ 0.16543388]]), matrix([[-0.45340747],
         [ 0.42411375],
         [ 0.02319054]]), matrix([[-0.35956512],
         [ 0.4262904 ],
         [-0.06178597]]), matrix([[-0.25957962],
         [ 0.42010482],
         [-0.10076008]]), matrix([[-0.1727498 ],
         [ 0.41132612],
         [-0.1090484 ]]), matrix([[-0.10595548],
         [ 0.40401329],
         [-0.10017001]]), matrix([[-0.05907209],
         [ 0.39930121],
         [-0.08390875]]), matrix([[-0.02871266],
         [ 0.3967478 ],
         [-0

In [10]:
for i in ddp.xs:
    for j in i:
        j = j + 1e-4

In [16]:
for i in ddp.us:
    for j in i:
        j = j + 1e-4

In [17]:
ddp.us

[matrix([[0.75850714],
         [3.53195802]]), matrix([[-2.22082271],
         [ 1.94135449]]), matrix([[-3.28868346],
         [-0.05770897]]), matrix([[-2.82706126],
         [-1.4843291 ]]), matrix([[-1.63602261],
         [-2.08206944]]), matrix([[-0.36973173],
         [-1.94324835]]), matrix([[ 0.52259526],
         [-1.42243338]]), matrix([[ 0.93867587],
         [-0.84976513]]), matrix([[ 1.00176658],
         [-0.38974106]]), matrix([[ 0.87272466],
         [-0.08288319]]), matrix([[0.67193435],
         [0.08878387]]), matrix([[0.47119597],
         [0.16261256]]), matrix([[0.30466621],
         [0.17599561]]), matrix([[0.18199628],
         [0.15812663]]), matrix([[0.09959532],
         [0.1282183 ]]), matrix([[0.04865448],
         [0.09702717]]), matrix([[0.01983081],
         [0.06942655]]), matrix([[0.00535368],
         [0.04676498]]), matrix([[-0.00044902],
         [ 0.02855565]]), matrix([[-0.00139952],
         [ 0.01349726]])]

In [18]:
ddp2.solve(ddp.xs, ddp.us)

True

In [19]:
ddp2.iter

1